In [6]:
%load_ext rpy2.ipython
import numpy as np
import pandas as pd
import os
import h5py

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


We can use the filenames from the MinION to generate
some interesting statistics about our run!

The first thing we'll do is test to see if the QC held true.
If the initial QC says 1041 pores, then how many of those actually
produce at least one viable sequence?


In [7]:
PASS_DIR = ""

os.chdir(PASS_DIR)

fast5_files = [fast5_file for fast5_file in os.listdir(PASS_DIR)
              if fast5_file.endswith(".fast5")]
mux_dict = {}  # Dictionary with channel as key and
               # value as a list of each mux that went through.
               # We must initialise the dictionary first.
for channel_num in range(1, 513): # numbers 1 to 512 inclusive
    mux_dict[channel_num] = []  # Initialise value to be a list.

for fast5file in fast5_files:
    channel = fast5file.split('_')[-3]  # 'channel_512'
    channel_num = channel.replace('ch', "")  # '512'
    read = fast5file.split('_')[-2]  # 'read_23042'
    read_num = read.replace("read", "")  # '23042'
    # Open fast5 file
    f = h5py.File(fast5file)
    mux_path = "/Raw/Reads/Read_" + str(read_num)
    mux = f[mux_path].attrs.values()[2]
    mux_dict[channel_num].append(mux)  # adds this mux to the list for
                                       # this given channel.
    

# Now we convert the mux_dict into a dataframe 
# so it can be imported into R.
df = pd.DataFrame(mux_dict)

OSError: [Errno 2] No such file or directory: ''

In [ ]:
R%% -i df -o mux_stats # imports the dataframe into r.
# This will be imported as a list of lists. 
# We cannot import as a regular
# dataframe as each list is of different lengths.

THRESHOLD = 1;  # How many pass reads need to pass through the pore
                # to consider this a pass read.
    
total_viable_pores = 0;  # Initialise value then increment.

mux_per_channel = NULL  # array which we will append to then summarise
                      # for each channel. Will be given the number
                      # 0:4 for a given channel.
        
reads_per_pore = NULL  # for each channel a value of the number of reads
                          # produced by this channel.
    
for(channel in 1:length(df)){
    mux_list <- df[[channel]]
    mux_list_table <- table(mux_counts_table)
    mux_counts <- length(which(mux_list_table > THRESHOLD))
    read_count <- as.vector(table(mux_list))
    reads_per_pore <- c(reads_per_pore, )
    mux_per_channel <- c(mux_per_channel, mux_counts)
}

# Create a histogram using the mux_per_channel variable.
breaks = seq(-0.5, 4.5, by = 1)
xlab = "Number of pores used in channel."
ylab = "Number of channels."
main = "Histogram of the number of pores used."
hist(mux_per_channel, breaks=breaks,
    xlab=xlab, ylab=ylab, main=main)

# Create a log histogram of the number of pass reads
# each channel produced.


mux_stats = table(mux_per_channel) # Return this value so we can print
# a final QC using python.